In [1]:
# Read the dataset description
from pandas import read_csv


tagged = read_csv('C:/Users/wang/Desktop/kaggle/train.csv')

In [2]:
df_tagged = read_csv('C:/Users/wang/Desktop/kaggle/train.csv')
tagged={}
for i in range(0,len(df_tagged)):
    tagged[df_tagged.iloc[i,0]] = df_tagged.iloc[i,1]
    
df_submit = read_csv('C:/Users/wang/Desktop/kaggle/sample_submission.csv')
submit = list(df_submit.iloc[:,0])
len(tagged), len(submit), list(tagged.items())[:5], submit[:5]

(9850,
 15610,
 [('00022e1a.jpg', 'w_e15442c'),
  ('000466c4.jpg', 'w_1287fbc'),
  ('00087b01.jpg', 'w_da2efe0'),
  ('001296d5.jpg', 'w_19e5482'),
  ('0014cfdf.jpg', 'w_f22f3e3')],
 ['00029b3a.jpg',
  '0003c693.jpg',
  '000bc353.jpg',
  '0010a672.jpg',
  '00119c3f.jpg'])

In [3]:
# Determise the size of each image
from os.path import isfile
from PIL import Image as pil_image
import sys
sys.path.append('C:/Users/wang/Desktop/kaggle/tqdmmaster')
from tqdm import tqdm_notebook
location_train = 'C:/Users/wang/Desktop/kaggle/train/'
location_test = 'C:/Users/wang/Desktop/kaggle/test/'
def expand_path(path):
    if isfile(location_train + path):
        
        return location_train + path
    if isfile(location_test + path):
        
        return location_test + path
    
p2size = {}
for p in tqdm_notebook(list(tagged.keys())+submit):
    p2size[p] = pil_image.open(expand_path(p)).size
    

HBox(children=(IntProgress(value=0, max=25460), HTML(value='')))

In [4]:
from imagehash import phash

join = list(tagged.keys()) + submit
p_hash={}


# Compute phash for each image in the training and test set.
for image_name in tqdm_notebook(join):
    image = pil_image.open(expand_path(image_name))
    img = str(phash(image))
    p_hash[image_name] = img


# Given a phash value, find the associated image
hash_image={}

for img_name, img_hash in p_hash.items():
    if img_hash not in hash_image:
        hash_image[img_hash] = []
    if img_name not in hash_image[img_hash]:
        hash_image[img_hash].append(img_name)

            
hash_names = list(hash_image.keys())
    

HBox(children=(IntProgress(value=0, max=25460), HTML(value='')))

In [66]:
import numpy as np

def match(h1,h2):
    for p1 in hash_image[h1]:
        for p2 in hash_image[h2]:
            i1 = pil_image.open(expand_path(p1))
            i2 = pil_image.open(expand_path(p2))
            if i1.mode != i2.mode or i1.size != i2.size: return False
            a1 = np.array(i1)
            a1 = a1 - a1.mean()
            a1 = a1/np.sqrt((a1**2).mean())
            a2 = np.array(i2)
            a2 = a2 - a2.mean()
            a2 = a2/np.sqrt((a2**2).mean())
            a  = ((a1 - a2)**2).mean()
            if a > 0.1: return False
    return True


hash_names_match = {}
for i,h1 in enumerate(hash_names):
    for h2 in hash_names[:i]:
        if (h1-h2 <=6 and h1-h2 >= -6) or match(h1,h2):
            s1 = str(h1)
            s2 = str(h2)
            
            hash_names_match[s2] = s1

hash_image_new = {}
for i,j in hash_image.items():
    if i in hash_names_match:
        k = hash_names_match[i]
        if k not in hash_image_new:
            
            hash_image_new[k].append(j)
    if i not in hash_image_new:
        hash_image_new[i] = []
    else:
        hash_image_new[i].append(j)


KeyboardInterrupt: 

In [5]:
list(hash_image.items())[:5]

[('222236e5c3a4a1bd', ['00022e1a.jpg']),
 ('7d55b5f5f5fdf9e8', ['000466c4.jpg']),
 ('ebe0f4f43c7c1898', ['00087b01.jpg', '7c72d707.jpg']),
 ('2a8a9f97d351d554', ['001296d5.jpg']),
 ('d65656f4f4b5f4f4', ['0014cfdf.jpg', '89c94943.jpg'])]

In [6]:
# For each images id, select the prefered image
p2size = {}
for img_name in tqdm_notebook(join):
    p2size[img_name] = pil_image.open(expand_path(img_name)).size

best_h2p = {}
for key, value in hash_image.items():
    if len(hash_image[key])==1:
        best_h2p[key] = hash_image[key][0]
    else:
        best = hash_image[key][0]
        max_size = p2size[hash_image[key][0]]
        for i in range(1, len(hash_image[key])):
            if p2size[hash_image[key][i]] > max_size:
                best = hash_image[key][i]
                max_size = p2size[hash_image[key][i]]
        best_h2p[key] = best


    

HBox(children=(IntProgress(value=0, max=25460), HTML(value='')))

In [7]:
list(best_h2p.items())[:5],len(best_h2p)

([('222236e5c3a4a1bd', '00022e1a.jpg'),
  ('7d55b5f5f5fdf9e8', '000466c4.jpg'),
  ('ebe0f4f43c7c1898', '00087b01.jpg'),
  ('2a8a9f97d351d554', '001296d5.jpg'),
  ('d65656f4f4b5f4f4', '0014cfdf.jpg')],
 20931)

In [159]:
import random
from keras import backend as K
from keras.preprocessing.image import img_to_array,array_to_img
from scipy.ndimage import affine_transform
from keras.preprocessing.image import ImageDataGenerator

img_shape    = (384,384) # The image shape used by the model


In [240]:
def transformation(p, augment):
    """
    @param p : the name of the picture to read
    @param augment: True/False if data augmentation should be performed
    @return a numpy array with the transformed image
    """
    # Generate the transformation matrix
    
    image_gen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rescale=1./255,
        rotation_range=30,
        #width_shift_range=.55,
        #height_shift_range=.55,
        horizontal_flip=True,
        vertical_flip=True,
        zoom_range = 0.5)

    # Read the image, transform to black and white and comvert to numpy array
    img   = pil_image.open(expand_path(p)).convert('L').resize(img_shape)
    img   = img_to_array(img)
    img = img.reshape((1,384,384,1)).astype('float32')
    generator = image_gen.flow(img, batch_size = 1)
    if augment:
        img_list=[]
        i=0
        for batch in generator:
            print(batch.shape)
            img_list.append(batch)
            i+=1
            if i >=4:
                break
        return img_list
                

    else:
        return img
    
    

def read_for_training(p):
    """
    Read and preprocess an image with data augmentation (random transform).
    """
    return transformation(p, True)

def read_for_validation(p):
    """
    Read and preprocess an image without data augmentation (use for testing).
    """
    return transformation(p, False)


img = list(tagged.keys())[6]

img2 = read_for_training(img)
img3 = read_for_validation(img)



C:\Users\wang\Anaconda3\lib\site-packages\keras\preprocessing\image.py:500: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\wang\Anaconda3\lib\site-packages\keras\preprocessing\image.py:508: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


(1, 384, 384, 1)
(1, 384, 384, 1)
(1, 384, 384, 1)
(1, 384, 384, 1)


In [252]:
img_shape    = (384,384,1) 

In [253]:
from keras import regularizers
from keras.optimizers import Adam
from keras.engine.topology import Input
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, Flatten, GlobalMaxPooling2D, Lambda, MaxPooling2D, Reshape
from keras.models import Model

def subblock(x, filter, **kwargs):
    x = BatchNormalization()(x)
    y = x
    y = Conv2D(filter, (1, 1), activation='relu', **kwargs)(y) # Reduce the number of features to 'filter'
    y = BatchNormalization()(y)
    y = Conv2D(filter, (3, 3), activation='relu', **kwargs)(y) # Extend the feature field
    y = BatchNormalization()(y)
    y = Conv2D(K.int_shape(x)[-1], (1, 1), **kwargs)(y) # no activation # Restore the number of original features
    y = Add()([x,y]) # Add the bypass connection
    y = Activation('relu')(y)
    return y

def build_model(lr, l2, activation='sigmoid'):

    ##############
    # BRANCH MODEL
    ##############
    regul  = regularizers.l2(l2)
    optim  = Adam(lr=lr)
    kwargs = {'padding':'same', 'kernel_regularizer':regul}

    inp = Input(shape=img_shape) # 384x384x1
    x   = Conv2D(64, (9,9), strides=2, activation='relu', **kwargs)(inp)

    x   = MaxPooling2D((2, 2), strides=(2, 2))(x) # 96x96x64
    for _ in range(2):
        x = BatchNormalization()(x)
        x = Conv2D(64, (3,3), activation='relu', **kwargs)(x)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 48x48x64
    x = BatchNormalization()(x)
    x = Conv2D(128, (1,1), activation='relu', **kwargs)(x) # 48x48x128
    for _ in range(4): x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 24x24x128
    x = BatchNormalization()(x)
    x = Conv2D(256, (1,1), activation='relu', **kwargs)(x) # 24x24x256
    for _ in range(4): x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 12x12x256
    x = BatchNormalization()(x)
    x = Conv2D(384, (1,1), activation='relu', **kwargs)(x) # 12x12x384
    for _ in range(4): x = subblock(x, 96, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 6x6x384
    x = BatchNormalization()(x)
    x = Conv2D(512, (1,1), activation='relu', **kwargs)(x) # 6x6x512
    for _ in range(4): x = subblock(x, 128, **kwargs)
    
    x             = GlobalMaxPooling2D()(x) # 512
    branch_model  = Model(inp, x)
    
    ############
    # HEAD MODEL
    ############
    mid        = 32
    xa_inp     = Input(shape=branch_model.output_shape[1:])
    xb_inp     = Input(shape=branch_model.output_shape[1:])
    x1         = Lambda(lambda x : x[0]*x[1])([xa_inp, xb_inp])
    x2         = Lambda(lambda x : x[0] + x[1])([xa_inp, xb_inp])
    x3         = Lambda(lambda x : K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x4         = Lambda(lambda x : K.square(x))(x3)
    x          = Concatenate()([x1, x2, x3, x4])
    x          = Reshape((4, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x          = Conv2D(mid, (4, 1), activation='relu', padding='valid')(x)
    x          = Reshape((branch_model.output_shape[1], mid, 1))(x)
    x          = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x          = Flatten(name='flatten')(x)
    
    # Weighted sum implemented as a Dense layer.
    x          = Dense(1, use_bias=True, activation=activation, name='weighted-average')(x)
    head_model = Model([xa_inp, xb_inp], x, name='head')

    
    from keras import regularizers
from keras.optimizers import Adam
from keras.engine.topology import Input
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, Flatten, GlobalMaxPooling2D, Lambda, MaxPooling2D, Reshape
from keras.models import Model

def subblock(x, filter, **kwargs):
    x = BatchNormalization()(x)
    y = x
    y = Conv2D(filter, (1, 1), activation='relu', **kwargs)(y) # Reduce the number of features to 'filter'
    y = BatchNormalization()(y)
    y = Conv2D(filter, (3, 3), activation='relu', **kwargs)(y) # Extend the feature field
    y = BatchNormalization()(y)
    y = Conv2D(K.int_shape(x)[-1], (1, 1), **kwargs)(y) # no activation # Restore the number of original features
    y = Add()([x,y]) # Add the bypass connection
    y = Activation('relu')(y)
    return y

def build_model(lr, l2, activation='sigmoid'):

    ##############
    # BRANCH MODEL
    ##############
    regul  = regularizers.l2(l2)
    optim  = Adam(lr=lr)
    kwargs = {'padding':'same', 'kernel_regularizer':regul}

    inp = Input(shape=img_shape) # 384x384x1
    x   = Conv2D(64, (9,9), strides=2, activation='relu', **kwargs)(inp)

    x   = MaxPooling2D((2, 2), strides=(2, 2))(x) # 96x96x64
    for _ in range(2):
        x = BatchNormalization()(x)
        x = Conv2D(64, (3,3), activation='relu', **kwargs)(x)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 48x48x64
    x = BatchNormalization()(x)
    x = Conv2D(128, (1,1), activation='relu', **kwargs)(x) # 48x48x128
    for _ in range(4): x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 24x24x128
    x = BatchNormalization()(x)
    x = Conv2D(256, (1,1), activation='relu', **kwargs)(x) # 24x24x256
    for _ in range(4): x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 12x12x256
    x = BatchNormalization()(x)
    x = Conv2D(384, (1,1), activation='relu', **kwargs)(x) # 12x12x384
    for _ in range(4): x = subblock(x, 96, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 6x6x384
    x = BatchNormalization()(x)
    x = Conv2D(512, (1,1), activation='relu', **kwargs)(x) # 6x6x512
    for _ in range(4): x = subblock(x, 128, **kwargs)
    
    x             = GlobalMaxPooling2D()(x) # 512
    branch_model  = Model(inp, x)
    
    ############
    # HEAD MODEL
    ############
    mid        = 32
    xa_inp     = Input(shape=branch_model.output_shape[1:])
    xb_inp     = Input(shape=branch_model.output_shape[1:])
    x1         = Lambda(lambda x : x[0]*x[1])([xa_inp, xb_inp])
    x2         = Lambda(lambda x : x[0] + x[1])([xa_inp, xb_inp])
    x3         = Lambda(lambda x : K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x4         = Lambda(lambda x : K.square(x))(x3)
    x          = Concatenate()([x1, x2, x3, x4])
    x          = Reshape((4, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x          = Conv2D(mid, (4, 1), activation='relu', padding='valid')(x)
    x          = Reshape((branch_model.output_shape[1], mid, 1))(x)
    x          = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x          = Flatten(name='flatten')(x)
    
    # Weighted sum implemented as a Dense layer.
    x          = Dense(1, use_bias=True, activation=activation, name='weighted-average')(x)
    head_model = Model([xa_inp, xb_inp], x, name='head')

    ########################
    # SIAMESE NEURAL NETWORK
    ########################
    # Complete model is constructed by calling the branch model on each input image,
    # and then the head model on the resulting 512-vectors.
    img_a      = Input(shape=img_shape)
    img_b      = Input(shape=img_shape)
    xa         = branch_model(img_a)
    xb         = branch_model(img_b)
    x          = head_model([xa, xb])
    model      = Model([img_a, img_b], x)
    model.compile(optim, loss='binary_crossentropy', metrics=['binary_crossentropy', 'acc'])
    return model, branch_model, head_model

model, branch_model, head_model = build_model(64e-5,0)

In [255]:
branch_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 384, 384, 1)   0                                            
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 192, 192, 64)  5248        input_2[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 96, 96, 64)    0           conv2d_2[0][0]                   
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 96, 96, 64)    256         max_pooling2d_1[0][0]            
___________________________________________________________________________________________

____________________________________________________________________________________________________
activation_8 (Activation)        (None, 24, 24, 256)   0           add_8[0][0]                      
____________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)   (None, 12, 12, 256)   0           activation_8[0][0]               
____________________________________________________________________________________________________
batch_normalization_29 (BatchNor (None, 12, 12, 256)   1024        max_pooling2d_4[0][0]            
____________________________________________________________________________________________________
conv2d_31 (Conv2D)               (None, 12, 12, 384)   98688       batch_normalization_29[0][0]     
____________________________________________________________________________________________________
batch_normalization_30 (BatchNor (None, 12, 12, 384)   1536        conv2d_31[0][0]         

In [256]:
head_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 512)           0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, 512)           0                                            
____________________________________________________________________________________________________
lambda_3 (Lambda)                (None, 512)           0           input_3[0][0]                    
                                                                   input_4[0][0]                    
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 512)           0           input_3[0][0]           

In [12]:
# Find all the whales associated with an image id. It can be ambiguous as duplicate images may have different whale ids.
h2ws = {}
new_whale = 'new_whale'
for p,w in tagged.items():
    if w != new_whale: # Use only identified whales
        h = p_hash[p]
        if h not in h2ws: h2ws[h] = []
        if w not in h2ws[h]: h2ws[h].append(w)
for h,ws in h2ws.items():
    if len(ws) > 1:
        h2ws[h] = sorted(ws)
len(h2ws)
#list(h2ws.items())[:5]

8402

In [17]:
# For each whale, find the unambiguous images ids.
w2hs = {}
for h,ws in h2ws.items():
    if len(ws) == 1: # Use only unambiguous pictures

        w = ws[0]
        if w not in w2hs: w2hs[w] = []
        if h not in w2hs[w]: w2hs[w].append(h)
for w,hs in w2hs.items():
    if len(hs) > 1:
        w2hs[w] = sorted(hs)
w2hs

{'w_e15442c': ['222236e5c3a4a1bd'],
 'w_1287fbc': ['1111515555551515',
  '1ef4f4fcad2d1c98',
  '505090024cccff2f',
  '675e1135c5a57574',
  '67e6b0b9adada72d',
  '6df0b8acacaccccd',
  '755515555555f5f4',
  '7b15c0e4f0f83417',
  '7d54d51507171717',
  '7d55b5f5f5fdf9e8',
  '8180b2f2d16dfefe',
  '9999787cfc9c1d1d',
  'b19908f213395bf1',
  'b2b393939296d0f6',
  'bedff426d5c495f5',
  'd14500d991a5a5ab',
  'db5320f0bcbd7c4c',
  'dfbb6084c0a1ac3c',
  'ebfd00f0bcad4cce',
  'edd55514d4f4f50d',
  'ef4fc50111d1f1b9',
  'f5858cf8fcfce1c0',
  'f9f11505f4fcbd2d',
  'fdfd54b57c5d4d4f',
  'fe63419060c9f8ac',
  'ff4440d4f5f5edfd',
  'ff5d555555d5f4bd',
  'ffd55555f4bd5d1c',
  'fffffff50080bcfc'],
 'w_da2efe0': ['af818090f0fcbcbd',
  'd3d1159d84e0f0f8',
  'ebe0f4f43c7c1898',
  'f60480ec7c5c5cdd',
  'fbd5d594bc7c7818',
  'fd755545a5785c5c',
  'ffd55555d5fdfd79',
  'fff555555595f5fd',
  'ffff55555585f878',
  'ffffc08080f0f474'],
 'w_19e5482': ['2a8a9f97d351d554'],
 'w_f22f3e3': ['d65656f4f4b5f4f4', 'f8f2f5

In [21]:
# Find the list of training images, keep only whales with at least two images.
import random
train = [] # A list of training image ids
for hs in w2hs.values():
    if len(hs) > 1:
        train += hs
random.shuffle(train)
train_set = set(train)
train_set

{'dc20e6faf4e6d01c',
 '2abecf477c268081',
 'fff55555353d3d7d',
 '00e9e9e979599999',
 'ffffffb05050d0d0',
 '939b0351d5d5f5f5',
 '0101050d17884021',
 'efb58581a1a1e9e9',
 'fffdfdfd454105a5',
 '282ae9714a3de10d',
 '01055c0544623447',
 'ffffffffc0f0f85c',
 'f1fdbf70f0f8aca0',
 '00000015214999a5',
 'd79ec4c4ec68ea0f',
 'ffffffffaf5555e9',
 '2e91141eada989b8',
 'fdfdd9555584a5a1',
 '7cfcb8a86878f8f9',
 'fdf5f333fd560080',
 'ffffffffa585a1a9',
 'fcf4f4f4f4f4d4fc',
 '005555575511932a',
 'f5f5f5c0fef579d5',
 'f575fcfcf4f4f5dd',
 '5a7647437b7a4aba',
 '0ce7c712a8b4e347',
 'ffdb51b5b5254569',
 '2545555501696939',
 '00205515a5e9e968',
 '6022de9b16f2f018',
 '702022a4263c7878',
 '8e5b60314bce2963',
 '5997377e7c498100',
 '5dd4083e0a687e9f',
 '002989899b9b3333',
 'ffd595a5a1e9f8f8',
 '01055595a5ada929',
 'bb31b143d3701311',
 'f47405a5f4387d7d',
 'a9f02dadbf838292',
 '5472607d909e1ea8',
 'fed650d4f5fd7d7d',
 'f4a0995b1bcc64b6',
 '5555d53a7313c64e',
 '00000000a0a92898',
 '2c3a1a4915015dd5',
 '3c75cb02a5a

In [30]:
import numpy as np
w2ts = {} # Associate the image ids from train to each whale id.
for w,hs in w2hs.items():
    for h in hs:
        if h in train_set:
            if w not in w2ts: w2ts[w] = []
            if h not in w2ts[w]: w2ts[w].append(h)
for w,ts in w2ts.items(): w2ts[w] = np.array(ts)
    
t2i = {} # The position in train of each training image id
for i,t in enumerate(train): t2i[t] = i


In [34]:
w2ts

{'w_1287fbc': array(['1111515555551515', '1ef4f4fcad2d1c98', '505090024cccff2f',
        '675e1135c5a57574', '67e6b0b9adada72d', '6df0b8acacaccccd',
        '755515555555f5f4', '7b15c0e4f0f83417', '7d54d51507171717',
        '7d55b5f5f5fdf9e8', '8180b2f2d16dfefe', '9999787cfc9c1d1d',
        'b19908f213395bf1', 'b2b393939296d0f6', 'bedff426d5c495f5',
        'd14500d991a5a5ab', 'db5320f0bcbd7c4c', 'dfbb6084c0a1ac3c',
        'ebfd00f0bcad4cce', 'edd55514d4f4f50d', 'ef4fc50111d1f1b9',
        'f5858cf8fcfce1c0', 'f9f11505f4fcbd2d', 'fdfd54b57c5d4d4f',
        'fe63419060c9f8ac', 'ff4440d4f5f5edfd', 'ff5d555555d5f4bd',
        'ffd55555f4bd5d1c', 'fffffff50080bcfc'], dtype='<U16'),
 'w_da2efe0': array(['af818090f0fcbcbd', 'd3d1159d84e0f0f8', 'ebe0f4f43c7c1898',
        'f60480ec7c5c5cdd', 'fbd5d594bc7c7818', 'fd755545a5785c5c',
        'ffd55555d5fdfd79', 'fff555555595f5fd', 'ffff55555585f878',
        'ffffc08080f0f474'], dtype='<U16'),
 'w_f22f3e3': array(['d65656f4f4b5f4f4', 'f8f2f5ac